In [1]:
import csv
import cv2
import json

In [2]:
def read_csv(csv_path):
    content = []

    with open(csv_path, 'r') as csvfile:
        reader = csv.reader(csvfile, quoting = csv.QUOTE_ALL)
        for row in reader: # each row is a list
            content.append(row)

    return content


def list_to_dict(lst):
    dct = {}
    for row in labels_ts:
        if row[1] in dct.keys():
            # there are some repititions in the csv labels
            # if the exact segment was previously present in dictionary, skip it
            flag = False
            for segm in dct[row[1]]:
                item = {'segment': [(row[2]), (row[3])]}
                str1 = str(item['segment'][0]) + str(item['segment'][1])
                str2 = str(segm['segment'][0]) + str(segm['segment'][1])
                if str1 == str2:
                    flag = True
                    continue
            if not flag:
                dct[row[1]].append({'segment': [float(row[2]), float(row[3])]})
        else:
            dct[row[1]] = [{'segment': [float(row[2]), float(row[3])]}]
    return dct


def frame_to_ts(frame_number, fps):
    return round(float(float(frame_number)/fps) , 2)

In [3]:
csvfile = '/usr/local/data02/zahra/datasets/Tempuckey/labels/ground_truth/tempuckey_ground_truth_labels.csv'
videos_path = '/usr/local/data02/zahra/datasets/Tempuckey/videos'
labels_frame = read_csv(csvfile)
labels_ts = []

In [4]:
for line in labels_frame:
    if line[0] == 'faceoff_ID':
        continue
    
    path_ = '{}/{}'.format(videos_path, line[1])
    cap = cv2.VideoCapture(path_)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    line_ts = [line[0], \
               line[1], \
               frame_to_ts(float(line[2]), fps), \
               frame_to_ts(float(line[3]), fps), \
               line[4]]
    
    labels_ts.append(line_ts)

In [5]:
labels_json = list_to_dict(labels_ts)

In [6]:
labels_json_annotation = {}
labels_json_annotation['version'] = 'VERSION 1.0'
labels_json_annotation['results'] = labels_json
labels_json_annotation['external_data'] = {}
json.dump(labels_json_annotation, open('/usr/local/data02/zahra/datasets/Tempuckey/labels/tempuckey_ground_truth_annotations.json','w'))